# Lab 6: Resampling & Model Selection

The goal of this lab is to give you the tools to:

 - Perform bootstrapping
 - Use Leave One Out Cross-Validation (LOOCV)
 - Perform feature selection with:
    - Best Subset Selection
    - Forward Stepwise Selection
    - Backward Stepwise Selection

In [1]:
# Our well-trusted libraries
import numpy as np
import pandas as pd

# Sci-kit learn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression,LinearRegression, Lasso
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score, cross_validate

# Statsmodels
import statsmodels.api as sm

# Visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# Iterator building blocks
# example: combinations('ABCD', 2) --> AB AC AD BC BD CD
from itertools import combinations

# Many were concerned with warnings. The short answer is that FutureWarning (most common)
# appears when a functionality is deprecated and will be replaced. Here's how to ignore them
# even if you should find a way to resolve them in a production environment.
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

Throughout this lab, we will use an extremely simple dataset as this helps with the intuition. In essence, we will try to determine whether a person will default (i.e., fail to make their interest or principal payments on a debt) with their account balance, income, and a student indicator. More precisely:

 - `default` (str, binary): Whether the individual defaulted
 - `student` (str, binary): Whether the individual is a student
 - `balance` (float, continuous): The individual's account balance
 - `income` (float, continuous): The individual's annual income

In [2]:
default = pd.read_csv('default.csv',index_col=0)


display(default.head())
print(default.shape, '\n')
default.value_counts('default')

,default,student,balance,income
1,No,No,729.526495,44361.625074
2,No,Yes,817.180407,12106.134700
3,No,No,1073.549164,31767.138947
4,No,No,529.250605,35704.493935
5,No,No,785.655883,38463.495879


(10000, 4) 



default
No     9667
Yes     333
dtype: int64

Let's turn the `default` and `student` into binary variables. We're only using best practices after all!

In [3]:
encoding_dict = {'Yes': 1,'No': 0}
for col in ['default', 'student']:
    default[col] = default[col].map(encoding_dict)

default.head()

,default,student,balance,income
1,0,0,729.526495,44361.625074
2,0,1,817.180407,12106.134700
3,0,0,1073.549164,31767.138947
4,0,0,529.250605,35704.493935
5,0,0,785.655883,38463.495879


Before we go into resampling methods, let's estimate the model with the original sample (i.e, `default`) to get a sense of the estimates of the coefficients as well as the standard errors associated with them.

In [5]:
X = default[['balance', 'income']]
X = sm.add_constant(X)

y = default['default']

display(X.head(), y.head())

results = sm.Logit(y, X).fit()
print(results.summary())

,const,balance,income
1,1.0,729.526495,44361.625074
2,1.0,817.180407,12106.134700
3,1.0,1073.549164,31767.138947
4,1.0,529.250605,35704.493935
5,1.0,785.655883,38463.495879


1    0
2    0
3    0
4    0
5    0
Name: default, dtype: int64

Optimization terminated successfully.
         Current function value: 0.078948
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:                default   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9997
Method:                           MLE   Df Model:                            2
Date:                Fri, 14 Feb 2025   Pseudo R-squ.:                  0.4594
Time:                        00:54:19   Log-Likelihood:                -789.48
converged:                       True   LL-Null:                       -1460.3
Covariance Type:            nonrobust   LLR p-value:                4.541e-292
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -11.5405      0.435    -26.544      0.000     -12.393     -10.688
balance        0.0056      0

Here, we are interested in the estimate and the standard error of each coefficient. We will now see how resampling methods perform while keeping this as our initial finding.

## Bootstrapping

Bootstrapping is a method for estimating sampling distributions of an estimator by resampling with replacement form the original sample. It is a flexible way to quantify uncertainty associated with an estimator or a model. Bootstrapping is especially useful because it allows us to mimic the process of selecting many observations from the population.

We use bootstrapping for estimating the distribution of a statistic (e.g. mean, variance) without using normality assumptions; in this lab, we are going to estimate hard to calculate standard errors (or confidence intervals). It is also used when the population is too small to generate a large sample.

The intuition behind the bootstrap is that we approximate the real distribution in the population by the original sample. This of course relies on the distribution in the original sample to approximate the population (which increases with n). Keep in mind: “Bootstrap confidence intervals are neither exact nor optimal, but aim instead for a wide applicability, combined with near-exact accuracy” (Efron and Hastie).

Throughout, we will use the following terminology when referring to the samples:

- **Original Sample:** The one sample of size `n` we have from the population
- **Subsample:** We draw `B` subsamples (of size `n`) __with replacement__ from the original sample. Choosing how many bootstrap replications to perform is subjective; infinity is ideal, but we will settle for “as many as possible”.

The bootstrap algorithm goes as follows:

1. Obtain the original sample from the population (size `n`)
2. Draw a subsample (with replacerment) from the original sample (size `n`)
3. Compute the estimate ($ \hat{\alpha}_{b}^{*} $) from the subsample
4. Repeat steps 2 and 3 `B` times
5. Compute $ SE_{B}(\hat{\alpha}) = \sqrt{\frac{1}{B-1} \sum_{b=1}^{B} (\hat{\alpha}_{b}^{*} - \bar{\alpha}_{b}^{*})^{2}} $ where $ \bar{\alpha}_{b}^{*} = \frac{\sum_{b=1}^{B} \hat{\alpha}_{b}^{*}}{B}$

![Bootstrap Algorithm](bootstrap_algorithm.png)


Anyways, let's get into the code to replicate the bootstrap algorithm we just looked at. 

Step 1 is already done as `default` is our original sample of length `n = 10000`. 

Step 2 is to draw a subsample of `n` observations (with replacement) from the original sample. To do this, we will create the function `get_indices` to randomly select `n` indicies with replacement from our df. 

In [8]:
def get_indices(data, n, seed):
    '''
    Generates a random subsample (i.e., its indices)
    with replacement consisting of n observations each.
    Inputs:
        - data (pd.Dataframe): data to sample from
        - n (int): number of observations in the sample
        - seed (int): seed to set in the random number grenerator
    Output:
        - indices (np.ndarray): array of indices forming
         the samples
    '''
    rng = np.random.default_rng(seed)    # allows you to set your seed
    indices = rng.choice(data.index,     # Use the dataset's indices as the input
                         int(n),         # Number of indices per sample
                         replace=True    # Draw samples with replacement
                        )
    return indices

    

Make sure to set your random seed; this ensures the data is randomized the same way everytime you call the function. 

In [10]:
# Example of making a call to the get_indices function
get_indices(default, 10, 23)

array([ 360, 6940, 4224, 6415, 2636, 1287, 6060, 1138, 1202, 6534])

Remember, there is replacement, meaning that an observation can occur 0, 1, or multiple times in the subsample. Here's proof that it works:

In [12]:
np.asarray(np.unique(get_indices(default, 10000, 45), return_counts=True)).T[-10:]

array([[ 9986,     1],
       [ 9987,     1],
       [ 9989,     3],
       [ 9991,     2],
       [ 9993,     1],
       [ 9995,     2],
       [ 9996,     2],
       [ 9997,     2],
       [ 9998,     1],
       [10000,     1]])

Step 3 is to estimate the coefficients with each subsample. We create a function `boot_fn` which performs the `logit` on each subsample in order to get the estimates and their standard errors.

In [13]:
# similar to boot.fn in the exercise
def boot_fn(data, index, constant=True, features=['balance','income'], target='default'):
    '''
    Runs a logistic regression (with a constant) on only the specified
    indices within the data (i.e., on a single subsample).

    Inputs:
        - data (pd.Dataframe): data to sample from
        - indices (np.ndarray): array of indices forming the samples
        - features (lst of str): the name of the features
        - target (str): the name of the target

    Output:
        - coefficients (lst of float): the coefficients
    '''
    X = data[features].loc[index]
    if constant:
        X = sm.add_constant(X)
    y = data[target].loc[index]
    
    lr = sm.Logit(y,X).fit(disp=0)
    coefficients = [lr.params[0], lr.params[1], lr.params[2]]

    return coefficients

In [14]:
# Example of making a call to the boot_fn function
intercept, coef_balance, coef_income = boot_fn(default, get_indices(default, 10000, 45))
print(f'Coefficients of a single subsample:\n\tIntercept:\t{round(intercept, 2)}\n\tBalance:\t{round(coef_balance, 2)}\n\tIncome:\t\t{round(coef_income, 2)}')

Coefficients of a single subsample:
	Intercept:	-10.84
	Balance:	0.01
	Income:		0.0


In step 4, we want to repeat steps 2 and 3 `B` number of times to create many subsamples and estimate their coefficients.  
In step 5, we want to compute the mean and standard deviation based on an average of all the estimates.   
We will do both in the `boot` function.

# How long will this take to run?

In [15]:
import time

start_time = time.time()
#I want to take a 5 second nap
time.sleep(5)

end_time = time.time()

print(end_time - start_time)


5.005131721496582


In [16]:
def boot(data, func, B):
    '''
    Executing a bootstrap over B subsamples
    to estimate the (mean of) coefficients
    and their associated standard errors.

    Inputs:
        - data (pd.Dataframe): data to sample from
        - func (fn): function executing the regression
        - B (int): number of subsamples
    
    Ouput:
        - restults (dict of dicts): bootstrapped coefficients
            and the associated standard errors
    '''
    # Step 4
    coef_intercept = []
    coef_balance = []
    coef_income = []

    coefs = ['intercept', 'balance', 'income']
    output = {coef: [] for coef in coefs}
    for i in range(B):
        # set new seed (=i) every time you get a new subsample
        reg_out = func(data, get_indices(data, len(data), i))
        for i, coef in enumerate(coefs):
            output[coef].append(reg_out[i])

    # Step 5
    results = {}
    for coef in coefs:
        results[coef] = {
            'estimate': np.mean(output[coef]),
            'std_err': np.std(output[coef])
        }
    
    return results

Essentially, the three functions we created are the building blocks to performing the bootstrap algorithm. 

Now, we can finally run our own bootstrap on the data we have! 

In [17]:
start_time = time.time()

results = boot(default, boot_fn, 1000)
for i, x in results.items():
    print(f"{i.capitalize()}:\n\tEstimate: {x['estimate']}\n\tStandard Error: {x['std_err']}")

end_time = time.time()

print(end_time - start_time)


Intercept:
	Estimate: -11.599221788148167
	Standard Error: 0.44076600858304354
Balance:
	Estimate: 0.005675350638457242
	Standard Error: 0.00023101506115923834
Income:
	Estimate: 2.1092285173232854e-05
	Standard Error: 4.836558867887689e-06
8.662756204605103


We get values that are very similar to what we expected (i.e., from the logistic regression on the original sample). However, by using Bootstrap, we reduce the risk of overfitting and improve the stability of our machine learning algorithms.

#### Side Bar:

In [18]:
# the help() function will return the docstring of your function; 
# you can remind yourself what a function does at any point throughout your code
help(get_indices)

Help on function get_indices in module __main__:

get_indices(data, n, seed)
    Generates a random subsample (i.e., its indices)
    with replacement consisting of n observations each.
    Inputs:
        - data (pd.Dataframe): data to sample from
        - n (int): number of observations in the sample
        - seed (int): seed to set in the random number grenerator
    Output:
        - indices (np.ndarray): array of indices forming
         the samples



# Leave One Out Cross-Validation


## Note that we could also use log loss instead of the Brier score (which in a binary outcome scenario is equivalent to MSE). 

We already went over cross-validation in previous labs. However, we have yet to cover its most extreme version; Leave One Out Cross Validation (LOOCV).

As its name suggests, LOOCV is simply cross validation where the test set contains a single observation. There are a few pros and cons as to why you would choose to use cross-validation:

- **Pros:**
    - Low Upward Bias: because you only lose one observation when fitting the model on the training data
    - Final Test Error Not Variable: The test error will be the same every time you run LOOCV on the dataset because it goes through the same validation process every time. 
- **Cons:**
    - Computationally Costly: Requires estimating the model `n` different times
    - Prone to higher variance (especially when n is small)

In [27]:
squared_errors = []

#How will the random seed affect LOOCV?
# np.random.default_rng(5)
np.random.default_rng(1)
# np.random.default_rng(55)

# Iterate over the entire dataset one observation at a time
print('Iternation Milestones:...')
for i in default.index:
    # All except observation i is our training set
    train = default.iloc[default.index != i,:]
    # Only observation i is our test set
    test = default.iloc[default.index == i,:]

    # Fit the model and gather the squared error
    ols = LinearRegression().fit(train[['balance', 'income']], train['default'])
    test_predicted = ols.predict(test[['balance', 'income']])
    test_actual = test[['default']]
    squared_error = np.power(test_predicted - test_actual, 2)

    # Store the squared error
    squared_errors.append(squared_error.values[0][0])

    if i % 1000 == 0:
        print(i)

print(f'\n{squared_errors[:10]}\n')

# From the squared errors, get the Mean Squared Error (MSE)
print(f'MSE using LOOCV: {round(np.mean(squared_errors), 5)}')

Iternation Milestones:...
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000

[0.0005927380823778704, 0.00044313658761178823, 0.004082543662994585, 3.6509339069982594e-05, 0.0008426104370810332, 0.0010518241573572432, 0.0007861782470870673, 0.000504295884156107, 0.006093171030518981, 0.006208461720247789]

MSE using LOOCV: 0.02824


Notice how certain errors are way larger than others? Those are outliers, but they average out over the size of dataset so we don't need to give them much attention.

We can compare our results from LOOCV to regular KFold CV. An alternative way of doing k-fold cross validation is with the `cross_val_score` that is also in the sci-kit learn library. Here is how to use it with shuffling/randomizing of the data.

In [23]:
from sklearn.model_selection import KFold

X = default[['balance','income']]
Y = default['default']
ols = LinearRegression()

cv = KFold(5,              # Number of folds
           shuffle=True,   # Randomizes observations into folds if true
           random_state=9  # Affects how randomization occurs
          )  

cv_scores = cross_val_score(ols,      # Specified model
                            X,        # Features
                            Y,        # Target
                            cv = cv,  # Number of folds
                            scoring = 'neg_mean_squared_error'  # Metric
                           )

print(cv_scores)

print(np.mean(-cv_scores))

[-0.02271776 -0.03038288 -0.02430095 -0.03140294 -0.0325379 ]
0.028268485556214168


What if we tried to perform LOOCV on a smaller dataset and compare to the performance of KFold CV? 

To simulate this, we will randomly select smaller dataframes and perform LOOCV to compare how LOOCV changes as n increases. To do this, we can reuse our get_indices function from before.

In [24]:
#Compare the test errors from KFoldCV to LOOCV over different n size datasets
for n in [1000, 2500, 5000, 7500, 10000]:
    # Subset datasets of different sized n
    n_indices = get_indices(default, n, 42)
    subset = default.loc[n_indices].reset_index(drop=True)

    # Use the same code to get the LOOCV
    squared_errors = []
    for i in subset.index:
        # All except observation i is our training set
        train = subset.iloc[subset.index != i,:]
        # Only observation i is our test set
        test = subset.iloc[subset.index == i,:]

        # Fit the model and gather the squared error
        ols = LinearRegression().fit(train[['balance', 'income']], train['default'])
        test_predicted = ols.predict(test[['balance', 'income']])
        test_actual = test[['default']]
        squared_error = np.power(test_predicted - test_actual, 2)

        # Store the squared error
        squared_errors.append(squared_error.values[0][0])  
    print('MSE using LOOCV using n = '+ str(n) + f': {round(np.mean(squared_errors), 5)}')


    # Use the same code to get the error from the KFold CV
    X = subset[['balance','income']]
    Y = subset['default']
    ols = LinearRegression()

    cv = KFold(5, shuffle=True, random_state=9)  

    cv_scores = cross_val_score(ols, X, Y, cv = cv, scoring = 'neg_mean_squared_error')

    print('MSE using KFold CV using n = '+ str(n) + f': {round(np.mean(-cv_scores), 5)}')
    

MSE using LOOCV using n = 1000: 0.02797
MSE using KFold CV using n = 1000: 0.02787
MSE using LOOCV using n = 2500: 0.02738
MSE using KFold CV using n = 2500: 0.02737
MSE using LOOCV using n = 5000: 0.02875
MSE using KFold CV using n = 5000: 0.02874
MSE using LOOCV using n = 7500: 0.0282
MSE using KFold CV using n = 7500: 0.02823
MSE using LOOCV using n = 10000: 0.02846
MSE using KFold CV using n = 10000: 0.02846


From our mini experiment, we see that the KFold CV performs almost equally to LOOCV when applied to the `default` dataset. In general, we prefer LOOCV in instances where n is small because it allows our model to fit over more training samples.

## Best Subset Selection


## mlextend's Exhaustive Feature Selection

In [26]:
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS

X = default[['balance', 'income', 'student']]
y = default['default']

model = LinearRegression()

efs = EFS(
    model,
    min_features=1,
    max_features=3,  
    scoring='r2',  
    cv=5,  
    n_jobs=-1 
)

efs.fit(X, y)

best_subset = max(efs.subsets_.values(), key=lambda x: x['avg_score']) 
best_features = best_subset['feature_names']  
best_adj_r2 = best_subset['avg_score']  

print(f"Best Model: Features = {best_features}, Adjusted R² = {round(best_adj_r2, 5)}")


Features: 7/7

Best Model: Features = ('balance', 'student'), Adjusted R² = 0.12292


We can see that the combination of features that best predict whether an individual will default is `student` and `balance`. This hints that `income` is an irrelevant variable when the others are given. This suggests that `income` is highly correlated with at least one of the other variables and thus only adds noise when predicting on the test set.

Anyways, now we know which features form the best subset. However, computing all possible combinations is not always feasible. As seen in class, the more features, the larger the data, and the more computationally intensive the model, the longer it takes to compute all possible combinations. Therefore, there is a need for other feature selections methods.

# sklearn's Sequential Feature Selector, First forwards

In [30]:
###################################
## Using sklearn's Sequential Feature Selector:  General Setup

default['CONSTANT'] = 1 

X = default[['balance', 'income', 'student']]
y = default['default']


ols = LinearRegression()

rs = 314

kfcv = KFold(5,               
             shuffle=True,    
             random_state=rs  
             )

### Forward Stepwise Selection

The idea of this model is that we want to add the variable that provides the biggest improvement to our model (the one that yields the smallest error). We start with a model that has a constant as its only feature and continue until we run out of variables or see an increase in our error rate.

Its algorithm is fairly intuitive:

1. For k = 0, let $ M_{0} $ denote the null model (no Xs)
2. For k = 1, ..., p:
    - Consider all p - k + 1 models that add one predictor to $ M_{k-1} $
    - Pick the best (smallest RSS or largest $ R^{2} $) of these models and call it $ M_{k} $
3. Select the single best (CV test error, $ C_{p} $, AIC, BIC, or adjusted $ R^{2} $) model among $ M_{0}$, ..., $ M_{p}$

In [53]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import KFold

from sklearn.metrics import mean_squared_error, make_scorer

rmse_scorer = make_scorer(mean_squared_error, greater_is_better=False, squared=True)  # RMSE scoring

sfs_forward = SequentialFeatureSelector(
    ols,
    n_features_to_select="auto",
    direction="forward",
    scoring=rmse_scorer,  # Use RMSE instead of R²
    cv=kfcv
)

# Fit the feature selector
sfs_forward.fit(X, y)

# Extract selected features
selected_features = X.columns[sfs_forward.get_support()].tolist()

print("\nBest Forward Stepwise Model by RMSE:")
print(selected_features)

best_model_rmse = -cross_val_score(ols, X[selected_features], y, cv=kfcv, scoring=rmse_scorer).mean()
print(f"\nBest Model RMSE: {round(best_model_rmse, 5)}")


Best Forward Stepwise Model by RMSE:
['balance']

Best Model RMSE: 0.02827


We see that forward stepwise selection chooses the model with `balance`. This disagrees with the "Best" subset selection.  Note there is no guarantee that we do find the best model from all possible combinations with forward stepwise selection.

### Backward Stepwise Selection
The idea of this model is that we want to remove the variable that is not adding predictive power to our model (one that we can remove without increasing our error). We start with a full model (using all of our variables) and drop variables until we see an increase in our error rate or end up with only a constant. The idea remains the exact same as in forward stepwise selection, but we start at a different point.

The algorithm goes as follows:

1. For k = p, let $ M_{p} $ denote the fully specified model (all Xs)
2. For k = p-1, ..., 0:
    - Consider all p - k + 1 models that remove one predictor from $ M_{k+1} $
    - Pick the best (smallest RSS or largest $ R^{2} $) of these models and call it $ M_{k} $
3. Select the single best (CV test error, $ C_{p} $, AIC, BIC, or adjusted $ R^{2} $) model among $ M_{0}$, ..., $ M_{p}$

In [52]:
sfs_backward = SequentialFeatureSelector(
    ols,
    n_features_to_select="auto",  # Automatically selects the best subset
    direction="backward",
    scoring=rmse_scorer,
    cv=kfcv
)

# Fit the feature selector
sfs_backward.fit(X, y)

# Extract selected features
selected_features = X.columns[sfs_backward.get_support()].tolist()

print("\nBest Backward Stepwise Model by RMSE:")
print(selected_features)

# Evaluate RMSE of the best model
best_model_rmse = -cross_val_score(ols, X[selected_features], y, cv=kfcv, scoring=rmse_scorer).mean()
print(f"\nBest Model RMSE: {round(best_model_rmse, 5)}")


Best Backward Stepwise Model by RMSE:
['balance', 'student']

Best Model RMSE: 0.02824


Preserve for posterity

In [54]:
def create_comb_betweenX(X):
    '''
    Returns a list with all possible combinations
    of a given list of features.

    Input:
        - X (lst of str): the features

    Output:
        - comb (lst of lst): All possible
          combinations
    '''
    interm = []
    for i in range(1, len(X)+1):
        interm.extend(list(combinations(X, i)))

    comb = []
    for i in interm:
        comb.append(list(i))

    return comb

In [55]:
comb = create_comb_betweenX(['student', 'balance', 'income'])
print(comb)

[['student'], ['balance'], ['income'], ['student', 'balance'], ['student', 'income'], ['balance', 'income'], ['student', 'balance', 'income']]


We can see that we have a list of all possible combinations that contain between 1 and `p` features. (`p = len(X.columns)`)

In [56]:
#Let's run one regression for every possible combination and store its value
comb_dict = {}

for i in comb: # For every combination
    X = default[i]
    Y = default['default']
    ols = LinearRegression()
    mse = np.mean(-1*cross_val_score(ols, 
                                     X, 
                                     Y, 
                                     cv = KFold(5, shuffle=True, random_state=9), 
                                     scoring = 'neg_mean_squared_error'))
    comb_dict[str(i)] = mse

display(comb_dict)
print(min(comb_dict, key=comb_dict.get))

{"['student']": 0.03217450950283521,
 "['balance']": 0.028303358251519306,
 "['income']": 0.03219876684675046,
 "['student', 'balance']": 0.028263423110388763,
 "['student', 'income']": 0.03217256077477485,
 "['balance', 'income']": 0.028268485556214168,
 "['student', 'balance', 'income']": 0.028261871785201197}

['student', 'balance', 'income']


We can see that the combination of features that best predict whether an individual will default is `student` and `balance`. This hints that `income` is an irrelevant variable when the others are given. This suggests that `income` is highly correlated with at least one of the other variables and thus only adds noise when predicting on the test set.

Anyways, now we know which features form the best subset. However, computing all possible combinations is not always feasible. As seen in class, the more features, the larger the data, and the more computationally intensive the model, the longer it takes to compute all possible combinations. Therefore, there is a need for other feature selections methods.

## Forward feature selection

In [57]:
# Add constant to dataframe
default['constant'] = 1 

# Specify target
Y = default['default']

# Variables to use in forward propagation
vars_left_add = ['student', 'balance', 'income'] 

# Regression type
ols = LinearRegression()

# Starting variables (constant initially)
current_vars = ['constant']

X = default[current_vars]
benchmark_error = np.mean(-1*cross_val_score(ols, X, Y,
                                             cv = KFold(5, shuffle=True, random_state=9),
                                             scoring = 'neg_mean_squared_error'))
print(' Initial run with only one var (constant term/only bias weight):', current_vars)
print('      Benchmark error:', benchmark_error)
print('')

# Keep adding the best variables (until no improvement can be made)
for iter in range(len(vars_left_add)):
    print('\033[1m'+ 'Iteration:', iter, '\033[0m')
    error_list = []
    # Iterate over all the variables left to add
    for var in vars_left_add:
        # Modify X according to current iteration
        X = default[current_vars + [var]]
        # Perform 5-fold CV to get errors
        error = np.mean(-1*cross_val_score(ols, X, Y,
                                           cv = KFold(5, shuffle=True, random_state=9),
                                           scoring = 'neg_mean_squared_error'))
        error_list.append(error)
        print(' Running model with:', current_vars + [var])
        print('      Error:', error)

    # Chose the smallest error
    min_error = min(error_list)
    chosen_col_index = error_list.index(min_error)

    # If our current smallest error is smaller than our previous error, than we add a variable
    if min_error < benchmark_error:
        print('          *** Variable selected:', vars_left_add[chosen_col_index])
        print('          *** Min error selected:', min_error)
        print('')
        # Add the variable that produced the smallest error to current_vars
        current_vars.append(vars_left_add[chosen_col_index])
        # Delete chosen variable from vars_left_add
        del vars_left_add[chosen_col_index] 
        # Update benchmark_error
        benchmark_error = min_error
    
    # Otherwise, we stop our model
    else:
        print('          \033[4m*** No variable was selected', '\033[0m')
        print('          *** Previous error rate (', benchmark_error,') is lower than smallest error rate of this iteration (', min_error ,')')
        print('          *** Break')
        break

print('')
print('Variables chosen for our model', current_vars)

 Initial run with only one var (constant term/only bias weight): ['constant']
      Benchmark error: 0.03220745625

Iteration: 0 
 Running model with: ['constant', 'student']
      Error: 0.03217450950283521
 Running model with: ['constant', 'balance']
      Error: 0.0283033582515193
 Running model with: ['constant', 'income']
      Error: 0.03219876684675046
          *** Variable selected: balance
          *** Min error selected: 0.0283033582515193

Iteration: 1 
 Running model with: ['constant', 'balance', 'student']
      Error: 0.028263423110388756
 Running model with: ['constant', 'balance', 'income']
      Error: 0.02826848555621416
          *** Variable selected: student
          *** Min error selected: 0.028263423110388756

Iteration: 2 
 Running model with: ['constant', 'balance', 'student', 'income']
      Error: 0.02826187178520121
          *** Variable selected: income
          *** Min error selected: 0.02826187178520121


Variables chosen for our model ['constant', '

We see that forward stepwise selection chooses the model with a constant, `balance`, `student` and `income`. This coincides with the best subset selection we performed previously. However, there is no guarantee that we do find the best model from all possible combinations with forward stepwise selection.

# Backwards Selection

In [58]:
from sklearn.model_selection import KFold
# Add constant to dataframe
default['constant'] = 1 

# Specify the target
Y = default['default']

# Variables to remove iteratively
vars_left_to_drop = ['student', 'balance', 'income'] 

# Regression type
ols = LinearRegression()

# Starting variables (all)
current_vars = ['constant'] + vars_left_to_drop

X = default[current_vars]
benchmark_error = np.mean(-1*cross_val_score(ols, X, Y,
                                             cv = KFold(5, shuffle=True, random_state=9),
                                             scoring = 'neg_mean_squared_error'))
print(' Initial run with all vars:', current_vars)
print('      Benchmark error:', benchmark_error)
print('')


# Keep removing the worst variables (until no improvement can be made)
for iter in range(len(vars_left_to_drop)):
    print('\033[1m'+ 'Iteration:', iter, '\033[0m')
    error_list = []
    # Iterate over all the variables left to remove
    for var in vars_left_to_drop:
        # Modify X according to current iteration
        vars_to_be_used = ['constant'] + [i for i in vars_left_to_drop if i != var]
        X = default[['constant'] + [i for i in vars_left_to_drop if i != var]]
        # Perform 5-fold CV to get errors
        error = np.mean(-1*cross_val_score(ols, X, Y,
                                           cv = KFold(5, shuffle=True, random_state=9),
                                           scoring = 'neg_mean_squared_error'))
        error_list.append(error)
        print(' Running model with:', vars_to_be_used)
        print('      Error:', error)

    # Chose the largest error
    min_error = min(error_list)
    chosen_col_index = error_list.index(min_error)

    # If our current smallest error is smaller than our previous error, then we drop the variable associated with it
    if min_error < benchmark_error:
        print('          *** Will drop:', vars_left_to_drop[chosen_col_index])
        print('          *** Min error selected:', min_error)
        print('          *** Chose the variable that generated the min error + was lower than previous error')
        print('')
        # Delete chosen variable from current_vars and vars_left_to_drop
        del current_vars[chosen_col_index + 1]
        del vars_left_to_drop[chosen_col_index]
        # Update benchmark_error
        benchmark_error = min_error
    
    # If not, we keep our model
    else:
        print('          \033[4m*** No variable was selected', '\033[0m')
        print('          *** Previous error rate (', benchmark_error,') is lower than smallest error rate of this iteration (', min_error ,')')
        print('          *** Break')
        break

print('')
print('Variables chosen for our model', current_vars)

 Initial run with all vars: ['constant', 'student', 'balance', 'income']
      Benchmark error: 0.028261871785201197

Iteration: 0 
 Running model with: ['constant', 'balance', 'income']
      Error: 0.02826848555621416
 Running model with: ['constant', 'student', 'income']
      Error: 0.03217256077477485
 Running model with: ['constant', 'student', 'balance']
      Error: 0.028263423110388746
          *** No variable was selected 
          *** Previous error rate ( 0.028261871785201197 ) is lower than smallest error rate of this iteration ( 0.028263423110388746 )
          *** Break

Variables chosen for our model ['constant', 'student', 'balance', 'income']
